<a href="https://colab.research.google.com/github/AvishrantsSh/Electrothon_3.0/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TEAM ENIGMA**

This file contains the different approaches we tried in this project.
We mainly focused on two major frameworks:
1. **Tensorflow**
2. **PyTorch**

In [ ]:
import tensorflow as tf
import numpy as np
from cv2 import *

In [ ]:
def conv_2d_layer(self,new_input,filter_shape,activate=tf.identity,stride=1,padding='SAME',name=None):
        with tf.variable_scope(name):
            W = tf.get_variable("W",shape=filter_shape,initializer=tf.random_normal_initializer(0., 0.005))
            b = tf.get_variable("b",shape=filter_shape[-1],initializer=tf.constant_initializer(0.))
            conv = tf.nn.conv2d(input=new_input,filters=w,strides=[1,stride,stride,1], padding=padding)
            bias = activate(tf.nn.bias_add(value=conv,bias=b))
        return bias

In [ ]:
def deconv_2d_layer(self,new_input,filter_shape,output_shape,activate=tf.identity,stride=1,padding='SAME',name=None):
        with tf.variable_scope(name):
            W = tf.get_variable("W",shape=filter_shape,initializer=tf.random_normal_initializer(0., 0.005))
            b = tf.get_variable("b",shape=filter_shape[-2],initializer=tf.constant_initializer(0.))
            deconv = tf.nn.conv2d_transpose(input=new_input,filters=W,output_shape=output_shape,strides=[1,stride,stride,1], padding=padding)
            bias = activate(tf.nn.bias_add(value=deconv,bias=b))
        return bias

In [ ]:
def new_fc_layer(self,new_input,output_size,name):
        shape = new_input.get_shape().as_list()
        dim = np.prod(shape[1:])
        x = tf.reshape(bottom,[-1, dim])
        input_size = dim
        with tf.variable_scope(name):
            W = tf.get_variable("W",shape=[input_size, output_size],initializer=tf.random_normal_initializer(0., 0.005))
            b = tf.get_variable("b",shape=[output_size],initializer=tf.constant_initializer(0.))
            fc = tf.nn.bias_add(value=tf.matmul(x,W),bias=b)
        return fc

In [ ]:
def channel_fc_layer(self, input, name):
        _,width,height,n_feat_map = input.get_shape().as_list()
        input_reshape = tf.reshape( input, [-1, width*height, n_feat_map] )
        input_transpose = tf.transpose( input_reshape, [2,0,1] )
        with tf.variable_scope(name):
            W = tf.get_variable("W",shape=[n_feat_map,width*height, width*height],initializer=tf.random_normal_initializer(0., 0.005))
            output = tf.batch_matmul(input_transpose, W)
        output_transpose = tf.transpose(output, [1,2,0])
        output_reshape = tf.reshape(output_transpose,[-1, height, width, n_feat_map])
        return output_reshape

In [ ]:
 def leaky_relu_func(self,new_input):
        return tf.nn.leaky_relu(new_input,alpha=0.1,)

In [ ]:
def batch_norm(self, new_input, is_train, var_epsilon=1e-8, name=None):
        new_input = tf.clip_by_value( new_input, -100., 100.)
        depth = new_input.get_shape().as_list()[-1]
        with tf.variable_scope(name):
            gamma = tf.get_variable("gamma",shape=[depth],initializer=tf.constant_initializer(1.))
            beta  = tf.get_variable("beta",shape=[depth],initializer=tf.constant_initializer(0.))
            batch_mean, batch_var = tf.nn.moments(new_input, [0,1,2], name='moments')
            exp_ma = tf.train.ExponentialMovingAverage(decay=0.5)

            def update():
                with tf.control_dependencies([exp_ma_op]):
                    return tf.identity(batch_mean), tf.identity(batch_var)

            exp_ma_op = exp_ma.apply([batch_mean, batch_var])
            exp_ma_mean, exp_ma_var = exp_ma.average(batch_mean), exp_ma.average(batch_var)
            mean, var = tf.cond(is_train,update,lambda:(exp_ma_mean, exp_ma_var) )

            normed = tf.nn.batch_normalization(new_input,mean=mean,variance=var,offset=beta,scale=gamma,variance_epsilon=epsilon)
        return normed

In [ ]:
def resize_conv_2d_layer(self, new_input, filter_shape, resize_scale=2, activation=tf.identity, padding='SAME', stride=1, name=None):
        width = new_input.get_shape().as_list()[1]
        height = new_input.get_shape().as_list()[2]
        new_input = tf.raw_ops.ResizeNearestNeighbor(new_input,size=[width*resize_scale, height*resize_scale])
        bias = self.conv_2d_layer(new_input, filter_shape, stride=1,name=name )
        return bias

In [ ]:
def generator( self, images, is_train ):  
        with tf.variable_scope('GEN'):
            conv1_1 = self.new_conv_layer(images, [3,3,3,32], stride=1, name="conv1_1" )
            conv1_1 = tf.nn.elu(conv1_1)
            conv1_2 = self.new_conv_layer(conv1_1, [3,3,32,32], stride=1, name="conv1_2" ) 
            conv1_2 = tf.nn.elu(conv1_2)
            conv1_stride = self.new_conv_layer(conv1_2, [3,3,32,32], stride=2, name="conv1_stride")
            conv2_1 = self.new_conv_layer(conv1_stride, [3,3,32,64], stride=1, name="conv2_1" ) 
            conv2_1 = tf.nn.elu(conv2_1)
            conv2_2 = self.new_conv_layer(conv2_1, [3,3,64, 64], stride=1, name="conv2_2" )
            conv2_2 = tf.nn.elu(conv2_2)
            conv2_stride = self.new_conv_layer(conv2_2, [3,3,64,64], stride=2, name="conv2_stride")
            conv3_1 = self.new_conv_layer(conv2_stride, [3,3,64,128], stride=1, name="conv3_1" ) 
            conv3_1 = tf.nn.elu(conv3_1)
            conv3_2 = self.new_conv_layer(conv3_1, [3,3,128, 128], stride=1, name="conv3_2" ) 
            conv3_2 = tf.nn.elu(conv3_2)
            conv3_3 = self.new_conv_layer(conv3_2, [3,3,128,128], stride=1, name="conv3_3" ) 
            conv3_3 = tf.nn.elu(conv3_3)
            conv3_4 = self.new_conv_layer(conv3_3, [3,3,128, 128], stride=1, name="conv3_4" )    
            conv3_4 = tf.nn.elu(conv3_4)
            conv3_stride = self.new_conv_layer(conv3_4, [3,3,128,128], stride=2, name="conv3_stride") 
            
            conv4_stride = self.new_conv_layer(conv3_stride, [3,3,128,128], stride=2, name="conv4_stride")
            conv4_stride = tf.nn.elu(conv4_stride)
            
            conv5_stride = self.new_conv_layer(conv4_stride, [3,3,128,128], stride=2, name="conv5_stride")
            conv5_stride = tf.nn.elu(conv5_stride)
            
            conv6_stride = self.new_conv_layer(conv5_stride, [3,3,128,128], stride=2, name="conv6_stride")
            conv6_stride = tf.nn.elu(conv6_stride)

            deconv5_fs = self.new_deconv_layer( conv6_stride, [3,3,128,128], conv5_stride.get_shape().as_list(), stride=2, name="deconv5_fs")
            debn5_fs = tf.nn.elu(deconv5_fs)
            
            skip5 = tf.concat([debn5_fs, conv5_stride], 3)
            channels5 = skip5.get_shape().as_list()[3]
  
            deconv4_fs = self.new_deconv_layer( skip5, [3,3,128,channels5], conv4_stride.get_shape().as_list(), stride=2, name="deconv4_fs")
            debn4_fs = tf.nn.elu(deconv4_fs)
            
            skip4 = tf.concat([debn4_fs, conv4_stride], 3)
            channels4 = skip4.get_shape().as_list()[3]
 
            deconv3_fs = self.new_deconv_layer( skip4, [3,3,128,channels4], conv3_stride.get_shape().as_list(), stride=2, name="deconv3_fs")
            debn3_fs = tf.nn.elu(deconv3_fs)
            
            skip3 = tf.concat([debn3_fs, conv3_stride], 3)
            channels3 = skip3.get_shape().as_list()[3]
   
            deconv2_fs = self.new_deconv_layer( skip3, [3,3,64,channels3], conv2_stride.get_shape().as_list(), stride=2, name="deconv2_fs")
            debn2_fs = tf.nn.elu(deconv2_fs)
            
            skip2 = tf.concat([debn2_fs, conv2_stride], 3)
            channels2 = skip2.get_shape().as_list()[3]
     
            deconv1_fs = self.new_deconv_layer( skip2, [3,3,32,channels2], conv1_stride.get_shape().as_list(), stride=2, name="deconv1_fs")
            debn1_fs = tf.nn.elu(deconv1_fs)    
            
            skip1 = tf.concat([debn1_fs, conv1_stride], 3)
            channels1 = skip1.get_shape().as_list()[3]
        
            recon = self.new_deconv_layer( skip1, [3,3,3,channels1],  images.get_shape().as_list(), stride=2, name="recon") 
        return recon

In [ ]:
def discriminator(self, images, is_train, reuse=None):
        with tf.variable_scope('DIS', reuse=reuse):
            conv1 = self.new_conv_layer(images, [4,4,3,64], stride=2, name="conv1" )
            bn1 = self.leaky_relu(self.batchnorm(conv1, is_train, name='bn1'))
            #bn1 = tf.nn.elu(conv1) 
            conv2 = self.new_conv_layer(bn1, [4,4,64,128], stride=2, name="conv2")
            bn2 = self.leaky_relu(self.batchnorm(conv2, is_train, name='bn2'))
            #bn2 = tf.nn.elu(conv2) 
            conv3 = self.new_conv_layer(bn2, [4,4,128,256], stride=2, name="conv3")
            bn3 = self.leaky_relu(self.batchnorm(conv3, is_train, name='bn3'))
            #bn3 = tf.nn.elu(conv3) 
            conv4 = self.new_conv_layer(bn3, [4,4,256,512], stride=2, name="conv4")
            bn4 = self.leaky_relu(self.batchnorm(conv4, is_train, name='bn4'))
            #bn4 = tf.nn.elu(conv4) 
            output = self.new_fc_layer( bn4, output_size=1, name='output')
        return output[:,0]       

In [ ]:
# Trying with PyTorch
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

In [ ]:
batchSize = 64 
imageSize = 64 

transform = transforms.Compose([transforms.Resize(imageSize), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),]) 

dataset = dset.CIFAR10(root = './data', download = True, transform = transform) 
dataloader = torch.utils.data.DataLoader(dataset, batch_size = batchSize, shuffle = True, num_workers = 2)

Extracting ./data/cifar-10-python.tar.gz to ./data


In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [ ]:
class G(nn.Module):

    def __init__(self):
        super(G, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(100, 512, 4, 1, 0, bias = False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias = False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias = False),
            nn.Tanh()
        )

    def forward(self, input):
        output = self.main(input)
        return output

In [ ]:
netG = G()
netG.apply(weights_init)

G(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

In [ ]:
class D(nn.Module):

    def __init__(self):
        super(D, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias = False),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(64, 128, 4, 2, 1, bias = False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(128, 256, 4, 2, 1, bias = False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(256, 512, 4, 2, 1, bias = False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace = True),
            nn.Conv2d(512, 1, 4, 1, 0, bias = False),
            nn.Sigmoid()
        )

    def forward(self, input):
        output = self.main(input)
        return output.view(-1)

In [ ]:
netD = D()
netD.apply(weights_init)

D(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)

In [ ]:
criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr = 0.0002, betas = (0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr = 0.0002, betas = (0.5, 0.999))


In [ ]:
for epoch in range(50):

    for i, data in enumerate(dataloader, 0):
        
        netD.zero_grad()
        
        real, _ = data
        input = Variable(real)
        target = Variable(torch.ones(input.size()[0]))
        output = netD(input)
        errD_real = criterion(output, target)
        
        noise = Variable(torch.randn(input.size()[0], 100, 1, 1))
        fake = netG(noise)
        target = Variable(torch.zeros(input.size()[0]))
        output = netD(fake.detach())
        errD_fake = criterion(output, target)

        errD = errD_real + errD_fake
        errD.backward()
        optimizerD.step()

        netG.zero_grad()
        target = Variable(torch.ones(input.size()[0]))
        output = netD(fake)
        errG = criterion(output, target)
        errG.backward()
        optimizerG.step()
        print('[%d/%d][%d/%d]' % (epoch, 50, i, len(dataloader),))
        if i % 100 == 0:
            vutils.save_image(real, '%s/real_samples.png' % "./results", normalize = True)
            fake = netG(noise)
            vutils.save_image(fake.data, '%s/fake_samples_epoch_%03d.png' % ("./results", epoch), normalize = True)